# Met Office WOW Exploration

In this notebook, data from WOW (The service acting as the point of access for crowd sourced recent and historical weather observations) is explored through various plots that visualize the data

### Prerequisites
To follow along with this notebook, data from WOW must be downloaded, the general steps for doing so are as follows:
* Create an account on WOW and log in
* Find desired sites and make sure the site reports desired parameter data during desired time selection viewable in the tables tab within the specific site view
* Download site CSV data using the export function (which export only appears once logged in)
* Place the downloaded data in the relevant directory

## Contents
- Explore Wow Data
  - Single Site
    - Load in Pre-Downloaded Data
    - Format and Explore the Data
      - Show Data Issue: Missing Observations
    - Plot the Data with Rolling Mean for Averages (which is an incorrect implementation due to missing data)
  - Multi Site
    - Load and Merge Data
      - Ensure no Duplicate Data has been Loaded
    - Plot Multi Site Information
    - Prune Data to Leave Desired Data
    - Plot Desired Data as a Whole (Messy)
    - Separate Data by Month per Site
    - Plot All Sites on a Separate plot for each Month (Cleaner)

In [1]:
# import required packages
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

In the case of this notebook, the single site data is data downloaded for the entire year of 2021 and the multi site data is downloaded from 4 months across the year of 2019. This information is used in generating the glob patterns below, and should be changed if the data is different to this.

In [2]:
# DEFINE NOTEBOOK SETTINGS
wow_download_dir = "data/WOW"
glob_pattern_for_single_site_data = wow_download_dir + "/*2021*.csv"
multi_site_glob_pattern = wow_download_dir + "/*2019*.csv"

# If this setting does not work, either disable the setting, or successfully get through-
# the datahub tutorial
PLOT_SITE_NAMES_MAP = True 
                            

if PLOT_SITE_NAMES_MAP:
    import cartopy.crs as ccrs
    import requests
    # clear cell outputs when private API keys are displayed
    from IPython.display import clear_output
    # read environment file
    env_vars = {}

    with open(".env") as f:
        for line in f:
            key, value = line.strip().split("=", 1)
            env_vars[key] = value

    SITE_ID = env_vars["SITE_SPECIFIC_CLIENT_ID"]
    SITE_SECRET = env_vars["SITE_SPECIFIC_CLIENT_SECRET"]
    spot_data_headers = {
        "X-IBM-Client-Id": SITE_ID,
        "X-IBM-Client-Secret": SITE_SECRET,
    }

## References

If material in the notebook was adapted from other source material for NOAA, link to it.

Also, add other material either cited in the text or that extends content in the notebook.